In [1]:
import bugzilla
import requests
import time
from datetime import datetime

In [2]:
pagure_owner_alias = requests.get("https://src.fedoraproject.org/extras/pagure_owner_alias.json").json()["rpms"]

In [3]:
orphaned = [p for p in pagure_owner_alias if "orphan" in pagure_owner_alias[p]]
orphaned[:10]

['8Kingdoms',
 '915resolution',
 '99soft-oss-parent',
 'AcetoneISO2',
 'Ajaxterm',
 'CPUFreqUtility',
 'CableSwig',
 'Canna',
 'CastPodder',
 'CodeAnalyst-gui']

In [4]:
bzapi = bugzilla.Bugzilla("bugzilla.redhat.com")

In [5]:
if not bzapi.logged_in:
    bzapi.interactive_login()

In [6]:
TRACKER = 1868278

query = bzapi.build_query(product="Fedora", include_fields=["flags", "assigned_to", "component", "creation_time"])
query["blocks"] = TRACKER
query["status"] = "NEW"
bugzillas = bzapi.query(query)
len(bugzillas)

63

In [7]:
RELENG = "releng@fedoraproject.org"
ORPHAN = "extras-orphan@fedoraproject.org"

In [8]:
pkgs_to_orphan = []
bugs_to_reassign = []

In [9]:
for bug in bugzillas:
    if bug.component in orphaned:
        if bug.assigned_to != ORPHAN:
            bugs_to_reassign.append(bug.id)
        continue
    needinfos = bug.get_flags("needinfo") or []
    for needinfo in needinfos:
        if needinfo.get('requestee') == bug.assigned_to:
            break
    else:
        # no needinfo on assignee
        continue
    bug_state = 0
    last_time = datetime.fromtimestamp(time.mktime(bug.creation_time.timetuple()))
    for comment in bug.getcomments():
        if comment["creator"] != RELENG:
            continue
        if not comment["text"].startswith("Dear Maintainer,"):
            continue
        comment_time = datetime.fromtimestamp(time.mktime(comment["creation_time"].timetuple()))
        delta = comment_time - last_time
        if bug_state == 0 and delta.days >= 7-1:
            last_time = comment_time
            bug_state = 1
        elif bug_state == 1 and delta.days >= 3*7-1:
            last_time = comment_time
            bug_state = 2
            break
    delta = datetime.now() - last_time
    if bug_state == 2 and delta.days >= 4*7-1:
        to_orphan = f"rpms/{bug.component}"
        pkgs_to_orphan.append(to_orphan)
        print(to_orphan)
        bugs_to_reassign.append(bug.id)

rpms/biber
rpms/python-pyriemann
rpms/rubygem-xml-simple
rpms/sugar-measure
rpms/Inventor
rpms/kjots
rpms/plasma-pass
rpms/paris-traceroute
rpms/python-aiomqtt
rpms/python-holidays
rpms/subfinder
rpms/rtags
rpms/xorg-x11-drv-nouveau
rpms/libclc
rpms/python-sshtunnel
rpms/emacs-auctex
rpms/aespipe
rpms/manifest-tool
rpms/papaki
rpms/koules
rpms/php-opencloud-openstack
rpms/xwota
rpms/icebreaker
rpms/rubygem-haml
rpms/rubygem-maruku
rpms/vagrant-lxc
rpms/python-sphinxcontrib-issuetracker
rpms/erlang-eper
rpms/htmldoc
rpms/emacs-haskell-mode
rpms/llvm10
rpms/php-composer-installers
rpms/php-guzzlehttp-psr7
rpms/php-typo3-phar-stream-wrapper
rpms/geronimo-osgi-support
rpms/puppet
rpms/oci-kvm-hook
rpms/opam
rpms/lmms
rpms/emacs-lookup
rpms/flannel
rpms/coffee-script
rpms/libgta
rpms/openvas-gsa
rpms/openvas-libraries
rpms/openvas-manager
rpms/openvas-scanner
rpms/grub2
rpms/python-transitions


In [10]:
print("https://bugzilla.redhat.com/buglist.cgi?bug_id=" + ",".join(str(b) for b in bugs_to_reassign))

https://bugzilla.redhat.com/buglist.cgi?bug_id=1905190,1917401,1923284,1923292,1923296,1923316,1923317,1923329,1923330,1923332,1923334,1923338,1923340,1923345,1923365,1923372,1923373,1923400,1923418,1923424,1923427,1923428,1923432,1923437,1923449,1923450,1923475,1923477,1923506,1923520,1923557,1923588,1923606,1923617,1923618,1923619,1923631,1923634,1923637,1923645,1923652,1923676,1923682,1923684,1923686,1923687,1923702,1923703,1923704,1923705,1924713,1926826
